In [ ]:
import pandas as pd
import numpy as np

# import sklearn

In [ ]:
# basic
import random
# import json
# import re
import pickle

# vizu
import matplotlib.pyplot as plt
# import seaborn as sns


# modelling
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# from sklearn.preprocessing import label_binarize
# from sklearn.pipeline import make_pipeline
# from sklearn import metrics
# from sklearn.metrics import confusion_matrix

# from sklearn.metrics import mean_squared_error

# from sklearn.neighbors import KNeighborsRegressor
# from sklearn import svm
# import xgboost as xgb

In [ ]:
myrandom = 42
np.random.seed(myrandom)
mu, sigma = 0, 1
feature_fake = np.random.normal(mu, sigma, 200)+2
target_fake = np.random.normal(mu, sigma, 200)*2
df = pd.DataFrame({'weight': feature_fake, 'price': target_fake})

In [ ]:
# joinded_scaled = gf.scale(joined[[kpi] + comp_raw], _min = 0, _max=1)

# scaler = StandardScaler().fit(df)
# df = pd.DataFrame(scaler.transform(df), columns=df.columns)

In [ ]:
df.plot(x='weight', y='price', style='o')
plt.title('weight vs weight')
plt.xlabel('weight')
plt.ylabel('price')
plt.show()

In [ ]:
predictors = ['weight']
target = 'price'
data_train, data_test, target_train, target_test = train_test_split(df[predictors], df[target], test_size=0.25, 
                                                                    random_state=myrandom)

In [ ]:
model=RandomForestRegressor(n_estimators=100, max_depth=3, random_state=myrandom)
model.fit(data_train, target_train)

In [ ]:
y_pred = model.predict(data_test)


In [ ]:
error_df = pd.DataFrame({'Actual': target_test, 'Predicted': y_pred})
error_df.plot(x='Actual', y='Predicted', style='o')
# plt.title('weight vs weight')
# plt.xlabel('weight')
# plt.ylabel('price')
# plt.show()

In [ ]:
import sklearn

In [ ]:
print("gyujtsed valahova a modellek eredmenyet!!!")

In [ ]:
print('Mean Absolute Error:', sklearn.metrics.mean_absolute_error(target_test, y_pred))
print('Mean Squared Error:', sklearn.metrics.mean_squared_error(target_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(sklearn.metrics.mean_squared_error(target_test, y_pred)))

In [ ]:
imp_df = imp_df(X_train.columns, perm.feature_importances_

In [ ]:
print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Validation Score: {:.2f}'.format(rf.score(X_train, y_train), 
                                                                                             rf.oob_score_,
                                                                                             rf.score(X_valid, y_valid)))

                                                                                             from sklearn.metrics import r2_score
from rfpimp import permutation_importances

def r2(rf, X_train, y_train):
    return r2_score(y_train, rf.predict(X_train))

In [ ]:
# variable importance (logit)

# print(model.intercept_)
# print(model.coef_)

logit = LogisticRegression(random_state=random_st, solver='lbfgs').fit(data_train, target_train)

def get_coefs_bar(fitted_clf, predictors, translator = None):
    imp_df = pd.DataFrame(fitted_clf.coef_)

    imp_df = imp_df.T
    imp_df['predictors'] = predictors
    if translator is not None:
        imp_df['predictors'] = pd.Series(predictors).replace(translator).replace({' ': '_'}, regex=True)

    imp_df = imp_df.set_index('predictors')
    ax = imp_df.plot.barh()
    ax.get_legend().remove()

get_coefs_bar(logit, predictors = predictors, translator = translat_comps)

In [ ]:
varimp = {}
for var,weight in zip(data_train.columns, model.feature_importances_):
    varimp[var] = weight
pd.DataFrame({'varimp': varimp}).sort_values('varimp').plot.barh()

In [ ]:
# variable importance (XGBoost)
model = str(best)
pars = {}
model = best
mlf.get_shap_summary_plot(data_train, target_train, data_test, model, pars)
mlf.get_shap_force_plot(data_train, target_train, data_test, model, pars)